# Пытаемся научить модель генерировать шутки (ну или что-то близкое к шуткам хотя бы)

Мы будем работать с рускоязычной моделью `ruGPT3` от Сбера и библиотекой `transformers` для её дообучения от Hugging Face

In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device("cuda:0")

c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Дообучение модели ruGPt-3
Загружаем модель ruGPT-3 medium. Это русскоязычная модель на основе архитектуры OpenAI, которая была разработана и обучена командами SberDevices, SberAI и SberCloud на открытых данных википедии, художественной литературе, диалогах, программном коде.

In [3]:
model_name_or_path = "sberbank-ai/rugpt3medium_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(DEVICE)

In [4]:
!nvidia-smi


Wed Oct 19 15:34:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 511.81       Driver Version: 511.81       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   40C    P8    13W /  N/A |   1595MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Как происходит обучение
Обучающий текст нарезается на случайные куски, которые составляются в последовательности из 1024 (2048 у GPT-3) токенов разделяясь специальным `<|endoftext|>` символом. Во время обучения, модель учится предсказывать (классифицировать) каждый токен в последовательности один за другим при помощь `CrossEntropy Loss.`

Так как входная последовательность всегда заполнена до конца, padding не используется. Но во время инференса, длина входного текста может быть произвольной, поэтому надо явно указывать чем паддить оставшиеся позиции. По дефолту использутеся тот же `<|endoftext|>`.

## Обучающие данные
Будем учить GPT генерировать анекдоты. Пока возьмём небольшой набор данных.

In [43]:
text = """
- Блин! - сказал слон, наступив на колобка. 


- Позвони Лене!
- Вот еще, с ленью разговаривать...



- В Англии создан велосипед-майнер, способный генерировать криптовалюту.
- Отлично, ждем унитаз-майнер и диван-майнер.



- А скажите, нет ли у вас ещё книг этого автора?
- Вам он так понравился?
- Нет, но в его книге я нашёл 100 долларов!



- Ты где вчера была целый день?
- На МАКСе!
- Ну, и как Макс, хорош?
- Супер! Завтра опять на него собираюсь?



- Папа, а где наш старый утюг?
- Ты что, сынок, в бандиты подался?
- Нет, в коллекторы.



- Вован ! А, Вован!
- Чё Димон ?!
- А где твоя ПАНАМА ?



- Скажи, а почему первая группа крови с отрицательным резусом, такая редкость?
- Потому, что для вампиров она такой-же деликатес, как для нас чёрная икорка.



- Тебе кто-нибудь уже говорил, что ты - долбоеб?
- Нет.
- Жалеют!



- А ведь мощность политика можно измерить.
- Интересно... И в каких же единицах?
- В лошадиных силах.



- Мучачос, мы должны проиграть русским, я договорился с Путиным!
- Буэно, дон Фернандо, но мы же хотели победить на мундиале!
- Мундиаль - это мундиаль, а миллиард долларов - это миллиард долларов!



- Могу ли я рассчитывать на твоё молчание?
- Это вряд ли. Но на мою болтовню ты можешь смело положиться.



- Изверги, почто фараонов обидели?
- А не хер было Моисея прогонять.



- Моня, такое ощущение, что офисные хомячки залегли в спячку.
- Таки да, Изя! Наступило первое путинское похолодание.



- Вы знаете в Гос. Думе России стало одним мафиози меньше.
- Как? Неужели опять кого то убили?
- Да нет, просто Кобзон туда ходить перестал.



- Какое у тебя образование?
- Курсы кинологов заочно и по блату.
- Ты как раз нам нужен. Есть работа.
- И в каком кино я буду сниматься?



- Свет, а Свет! Скажи, тебе пиво какое нравится - темненькое или светленькое?
- Да, Свет, какое, темненькое или светленькое?
- МНЕ НРАВИТСЯ ВОДКА, НО ВАМ ЭТОГО НЕ ПОНЯТЬ!



- Смотрите - ЕДИНОРОГ !!!
- Петровичь - ЗАДОЛБАЛ ! Одень трусы !



- Слушай одолжи денег?
- А ты отдашь?
- Хочешь поклянусь.
- Ну да, хочу.
- Да, чтоб мне всю жизнь есть, только мясо.
- Ты чё веган что ли?



- В чем сила брат ?
- В Виагре брат!!!



- Говорят, что в 120 лет начинают расти новые зубы!
- Ага, и зовутся они деревьями.... на могилке.



- У меня на работе стол возле двери. А ты где на работе сидишь?
- В интернете.



- Сулико, при ребёнке я не могу чинить электропроводку, уводи мальчика из дома!
- Почему?
- Из-за ненормативной лексики.



- Это кто там гавкает?!
- С тобой, свинья, не гавкает, а томагавкает, президент Соединенных Штатов!



- Ты грубишь обычным клиентам и лебезишь перед богатыми! Это позорит нас!
- По другому у меня не получается, я же из семьи бывших советских официантов.



- Если деньги не пахнут, то зачем их отмывают?
- Может, потому что на воре шапка смердит?



- Какой у тебя Хирш, коллега?
- Моя жена довольна!
- А вот мой завлаб не доволен. Говорит, мал у тебя, Хирш, Петров.



- Сара. Почему допросы после 22.00 ззапрещены, а перговоры вести можно.
- Иося. Так они все по вашингтонскому времени жили.



- Ленечка, покажи зайчика.
- Машенька, покажи белочку.
- Вовочка, покажи президента.



- А где же Яша Рабинович?
- Он сегодня отпросился, проблемы в семье...
- Не слышал, чтобы так называлось похмелье.



- Пап, а что такое пиво?
- Это, Вовочка, такой отличный бензин - папу заправлять.



- Ты брюнетка, а подруга у тебя блондинка, что вас связывает?
- Отдых для мозгов!



- Мужчины должны лежать у твоих ног...
- Нет, лучше между.
- Тогда не лежать, а лизать. Хули толку с просто лежащего между ног мужчины?



- Девушка, а что это за атъебут у вас на шее?
- Как вы сказали?!?!
- Девушка, ви доучились до четвейтого куйса, и не знаете, что такое «атъебут»?



- Холмс, кто же не стоит за этим таинственным похищением унитазов?
- Это же элементарно, Ватсон! Злодей Парашенко, кто же ещё.



- Люда! Где мая зарплата?!
- Я не знаю...
- А откуда у тебя шуба?!
- Так! Не нервничай меня.



- Почему московское метро теперь строят китайцы?
- Саб Бян Нинг так решил.



- Такой полный человек должен покупать себе два билета
- Извините.



- И как это у тебя совести хватает врать мне прямо в глаза?
- И действительно! Глупо как-то. По уму врать нужно в уши.



- Нет, ну кто же так снимает трусики! Медленнее, эротичнее, завлекательнее!
- Простите, а вы точно гинеколог?



- Семён Маркович, как вы относитесь к возрождению романтизма?
- Простите, вы имеете в виду в деловом смысле или просто так?



- Сейчас пойду и обожрусь французскими булками - Что так проголодался ?
- Нет - выбирал подходящий шрифт.



- Унылая пора. Глаз разочарованье...
- Александр Сергеич! Ну, сколько можно все про экономику да политику?!



- Чего носы повесили, друзья?
- Да, радоваться нечему



- У меня на 8 марта не встал, пришлось жене кольцо дарить.
- А у меня встал и я сэкономил.



- Как там в России?
- Воруют, Владимир Владимирович.
- Ничего, Россия большая, все не разворуют.



- Ты знаешь, мой балбес наговорил по телефону на 500 баксов?
- А куда он звонил?
- В Японию в «Секс по телефону».
- Ха-ха, смешно! Ему дешевле было заказать девочек по телефону.
- Смешнее другое, мой балбес не знает ни одного языка.



- Тебе не жалко ослика в ролике ЛДПР?
- После того, как он разорался на Пугачеву - нет!



- Оказывается, украинская армия состоит сплошь из одних героев.
- Жаль, что в ней нет обычных солдат и офицеров.



- Кто останется в Украине после отделения востока и юга?
- Гомосеки и правосеки!



- Ну, ребята, кто может назвать других героев России?
- Вовочка: - Борис Ельцин! Он тоже завел россиян в дебри …



- Нёма, 31 декабря Зеленский позвонил Путину. Как ты думаешь, что он ему сказал?
- Асисяй?



- Что будет делать Россия, если США ужесточит против нее санкции?
- Мы откроем другую Америку.



- А поехали на море, на машине!
- Да ты чё? У нас пока не амфибия.



- Какая твоя самая крупная покупка в последнее время?
- Хз, стиралка наверное..
- Пивка три бутылки..
- Это большая покупка?
- Так ЛИТРОВЫХ же!...



- Ну что, все прибыли?
- Прибыли-то все, только в этом году у нас не прибыли, а убытки...



- Как вас постричь?
- Как и всегда, х*й пойми как.



- Ни кому, ни когда нельзя верить...
- Ну уж, Розенталю-то можно!



- Миша, если я тебе дам три шоколадные конфеты. Ты поделишься с сестрёнкой?
- Может, и поделюсь. Только мне всё, что внутри, а ей фантики.
- Алёна, а ты, сколько бы хотела конфет съесть одну или две?
- Три.



- Опять в дерьмецо вляпался. Вдруг подбегает сотрудник ФСБ, в штацком.
- Под «дерьмецом», вы кого имели в виду?
- Посмотрите, второй ногой я наступил на дерьмо. А вы кого имеете в виду?



- Что-то у нас плохо работает холодильник.
- Почему ты так решил?
- Ну смотри, масло 82%, сметана 20%, творог 9%, а кефир вообще 0% загрузки.



- Я хочу подать объявление о том, что у меня умерла теща.
- Хорошо. Какой будет текст?
- Простите, но у нас объявление должно содержать как минимум 10 слов.
- Ну, хорошо, напишите: «Умерла Куликова Нина Васильевна, ха-ха-ха, ха-ха-ха!»



- Слышал про Пермское вымирание?
- Слышал?! Да я здесь живу!
"""

В библиотеке transformers есть готовые инструменты для подготовки датасета и даталодера. На вход нужен всего лишь один `.txt` файл с обучающим текстом.

In [45]:
from transformers import TextDataset, DataCollatorForLanguageModeling

# Сохраним обучающие данные в .txt файл 
train_path = 'train_dataset.txt'
with open(train_path, "w", encoding='utf-8') as f:
    f.write(text)

# Создание датасета
train_dataset = TextDataset(tokenizer=tokenizer,file_path=train_path,block_size=64)
  
# Создание даталодера (нарезает текст на оптимальные по длине куски)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Loading features from cached file cached_lm_GPT2Tokenizer_64_train_dataset.txt [took 0.000 s]


## Training
Для файнтюнинга нам понадобится объект класса Trainer, который сделает всё грязную работу за нас (посылаем лучи добра Hugging Face). Далее нужно будет всего-навсего запустить `trainer.train()`

In [47]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=200, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    warmup_steps=10,# number of warmup steps for learning rate scheduler
    gradient_accumulation_steps=8, # to make "virtual" batch size larger
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None) # Optimizer and lr scheduler
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [48]:
torch.cuda.empty_cache()

In [49]:
trainer.train()

***** Running training *****
  Num examples = 36
  Num Epochs = 200
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 8
  Total optimization steps = 200
100%|██████████| 200/200 [04:54<00:00,  1.47s/it]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 200/200 [04:54<00:00,  1.47s/it]

{'train_runtime': 294.9521, 'train_samples_per_second': 24.411, 'train_steps_per_second': 0.678, 'train_loss': 0.04101671695709228, 'epoch': 199.89}


TrainOutput(global_step=200, training_loss=0.04101671695709228, metrics={'train_runtime': 294.9521, 'train_samples_per_second': 24.411, 'train_steps_per_second': 0.678, 'train_loss': 0.04101671695709228, 'epoch': 199.89})

## Сохраняем и загружаем дообученную модель

In [50]:
output_dir = 'jokes_rugpt3'

model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Configuration saved in jokes_rugpt3\config.json
Model weights saved in jokes_rugpt3\pytorch_model.bin
tokenizer config file saved in jokes_rugpt3\tokenizer_config.json
Special tokens file saved in jokes_rugpt3\special_tokens_map.json


('jokes_rugpt3\\tokenizer_config.json',
 'jokes_rugpt3\\special_tokens_map.json',
 'jokes_rugpt3\\vocab.json',
 'jokes_rugpt3\\merges.txt',
 'jokes_rugpt3\\added_tokens.json')

In [51]:
torch.cuda.empty_cache()


# Результат файнтюнинга
Ну давайте посмотрим что нам сгенерирует наша модель...

In [67]:
# Пример вероятностного сэмплирвоания с ограничением
text = "-Владимир харченко сдаёт линал!\n"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=2,
                        temperature=1.5,
                        top_p=0.9,
                        max_length=70,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)
print('-----------------------------------------------------------------------------')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



-Владимир харченко сдаёт линал!
- Хули толку с этого дерьма?



- Моня, такое ощущение, что офисные хомячки залегли в спячку.
- Таки да, Изя! Наступило первое путинское похолодание.



- Вы
-----------------------------------------------------------------------------


Ещё примерчик

In [73]:
# Пример вероятностного сэмплирвоания с ограничением
text = "-Валентин Зайцев очень любит олимпиадное программирование!\n"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=2,
                        temperature=1.5,
                        top_p=0.9,
                        max_length=70,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



-Валентин Зайцев очень любит олимпиадное программирование!
- А как ты сам стал программистом?
- Как и все, на диване.



- Я хочу подать на развод.
- Не вопрос, через два года поженимся.



- Ты брюнетка или блондинка?
- Сму
